In [54]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from pprint import pprint

In [72]:
training_data = pd.read_csv('training_v7.csv')
training_data = training_data.drop(['item_cnt_prev_4_mnth_mean','avg_item_price_3mnth_ago'],axis=1)
training_data

,item_cnt_mnth,shop_id,date_block_num,year,month,item_id,item_category_id,item_cnt_mnthly_mean,item_cnt_prev_3_mnth_mean,item_cnt_mnth_t-1,item_cnt_mnth_t-2,item_cnt_mnth_t-3,item_cnt_mnth_t-4,item_cnt_mnth_t-5,item_cnt_mnth_t-6
0,6.0,0,0,2013,1,32,40,8.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,3.0,0,0,2013,1,33,37,3.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0,0,2013,1,35,40,7.500000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0,0,2013,1,43,40,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,2.0,0,0,2013,1,51,57,2.500000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1609119,6.0,59,33,2015,10,22087,83,6.424242,3.333333,3.0,2.0,5.0,1.0,2.0,2.0
1609120,2.0,59,33,2015,10,22088,83,6.187500,5.000000,1.0,7.0,7.0,4.0,3.0,3.0
1609121,1.0,59,33,2015,10,22091,83,2.294118,1.000000,3.0,0.0,0.0,0.0,0.0,0.0
1609122,1.0,59,33,2015,10,22100,42,1.000000,0.333333,1.0,0.0,0.0,0.0,0.0,0.0


In [92]:
X = training_data.iloc[:,1:15].values
y = training_data.iloc[:,0].values
y = np.absolute(y)
feature_list = [c for c in training_data.columns 
                if c not in 'item_cnt_mnth']

In [93]:
X[0]

array([0.000e+00, 0.000e+00, 2.013e+03, 1.000e+00, 3.200e+01, 4.000e+01,
       8.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00])

In [94]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)

In [95]:
rand_forest = RandomForestRegressor(bootstrap=True,n_estimators=100, n_jobs=-1, random_state=42)
rand_forest.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=-1, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [96]:
importances = list(rand_forest.feature_importances_)
feature_importances = [(feature, round(importance, 2)) for feature, 
                       importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: item_cnt_mnthly_mean Importance: 0.6
Variable: item_cnt_mnth_t-1    Importance: 0.09
Variable: item_cnt_prev_3_mnth_mean Importance: 0.05
Variable: date_block_num       Importance: 0.04
Variable: month                Importance: 0.04
Variable: item_id              Importance: 0.04
Variable: shop_id              Importance: 0.03
Variable: item_category_id     Importance: 0.03
Variable: item_cnt_mnth_t-2    Importance: 0.02
Variable: item_cnt_mnth_t-5    Importance: 0.02
Variable: item_cnt_mnth_t-3    Importance: 0.01
Variable: item_cnt_mnth_t-4    Importance: 0.01
Variable: item_cnt_mnth_t-6    Importance: 0.01
Variable: year                 Importance: 0.0


In [97]:
y_pred = rand_forest.predict(X_test)

In [98]:
RMSE = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
print("RMSE:", RMSE)

RMSE: 5.2647358610357236


# Test Submission

In [99]:
test_sub = pd.read_csv('test_processed_v3.csv')

In [100]:
input_features = test_sub.iloc[:,1:15]
test_sub['item_cnt_month'] = rand_forest.predict(input_features)
test_sub['item_cnt_month'].clip(upper=20,lower=0,inplace=True)

In [103]:
test_sub[['ID','item_cnt_month']].to_csv('my_submission_v6.csv', index=False)

In [104]:
test_sub

,ID,shop_id,date_block_num,year,month,item_id,item_category_id,item_cnt_mnthly_mean,item_cnt_prev_3_mnth_mean,item_cnt_mnth_t-1,item_cnt_mnth_t-2,item_cnt_mnth_t-3,item_cnt_mnth_t-4,item_cnt_mnth_t-5,item_cnt_mnth_t-6,item_cnt_month
0,0,5,34,2015,11,5037,19,1.444444,1.333333,0,1,3,1,1,1,1.49
1,1,5,34,2015,11,5320,55,0.000000,0.000000,0,0,0,0,0,0,0.40
2,2,5,34,2015,11,5233,19,2.000000,1.666667,1,3,1,0,2,3,1.85
3,3,5,34,2015,11,5232,23,1.000000,0.333333,0,0,1,0,0,0,1.00
4,4,5,34,2015,11,5268,20,0.000000,0.000000,0,0,0,0,0,0,0.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214195,214195,45,34,2015,11,18454,55,1.800000,0.333333,1,0,0,0,0,0,1.26
214196,214196,45,34,2015,11,16188,64,0.000000,0.000000,0,0,0,0,0,0,0.04
214197,214197,45,34,2015,11,15757,55,1.166667,0.000000,0,0,0,0,0,0,1.07
214198,214198,45,34,2015,11,19648,40,0.000000,0.000000,0,0,0,0,0,0,0.19


In [105]:
test_sub['item_cnt_month'].max()

20.0

In [106]:
test_sub['item_cnt_mnthly_mean'].min()

0.0